# Kong

[Kong Gateway](https://konghq.com/) is a lightweight, fast, and flexible API gateway that can be
extended through huge ecosystem of [plugins and integrations](https://docs.konghq.com/hub/).

Kong is trusted for billions of transactions a day, by 700+ customers of all sizes across all industries.

## Kong AI Gateway

[Kong AI Gateway](https://konghq.com/products/kong-ai-gateway) delivers a suite of AI-specific plugins
on top of the API Gateway platform, enabling you to:

* Route a single consumer interface to multiple models, across many providers
* Load balance similar models based on cost, latency, and other metrics/algorithms
* Deliver a rich analytics and auditing suite for your deployments
* Enable semantic features to protect your users, your models, and your costs
* Provide no-code AI enhancements to your existing REST APIs
* Leverage Kong's existing ecosystem of authentication, monitoring, and traffic-control plugins

## Get Started

Kong AI Gateway exchanges inference requests in the OpenAI formats - thus you can easily and quickly
connect your existing LlamaIndex OpenAI adaptor-based integrations directly through Kong with no code changes.

You can target hundreds of models across the [supported providers](https://docs.konghq.com/hub/kong-inc/ai-proxy/),
all from the same client-side codebase.

### Create LLM Configuration

Kong AI Gateway uses the same familiar service/route/plugin system as the API Gateway product,
with a declarative setup that launches a complete gateway system configured from a single
YAML file.

Create your gateway YAML file, using the [Kong AI-Proxy Plugin](https://docs.konghq.com/hub/kong-inc/ai-proxy/),
in this example for:

* **OpenAI** backend and **GPT-4o** model

### Adjust Kong to Support LlamaIndex Authentication Headers

We also add a virtual consumer, with its own API key, that can later be audited, rate-limited, etcetera.

To support this in LlamaIndex, we need to configure Kong to receive the API Key header in the right location
and format.

Output this file to `kong.yaml`:

```yaml
_format_version: "3.0"

services:
  - name: ai
    url: https://localhost:32000

    routes:
      - name: openai-gpt4o
        paths:
          - "/gpt-4o"
        plugins:
          - name: ai-proxy
            config:
              route_type: llm/v1/chat
              model:
                provider: openai
                name: gpt-4o
              auth:
                header_name: Authorization
                header_value: "Bearer <OPENAI_KEY_HERE>"  # replace with your OpenAI key again

          # Now we add a security plugin at the "individual model" scope
          - name: key-auth
            config:
              key_names:
                - Authorization

# and finally a consumer with **its own API key**
consumers:
  - username: department-1
    keyauth_credentials:
      - key: "Bearer department-1-api-key"
```

### Launch the Gateway

Launch the Kong open-source gateway, loading this configuration YAML, with one command:

In [ ]:
docker run -it --rm --name kong-ai -p 8000:8000 \
    -v "$(pwd)/kong.yaml:/etc/kong/kong.yaml" \
    -e "KONG_DECLARATIVE_CONFIG=/etc/kong/kong.yaml" \
    -e "KONG_DATABASE=off" \
    kong:3.8

### Execute Your LlamaIndex Code

Now you can configure your LlamaIndex client code to point to Kong.

First, load the LlamaIndex SDK into your Python dependencies, then execute the sample code:

In [ ]:
% pip install llama-index-llms-openai llama-index

and run a simple chat, overriding the OpenAI URL to point to Kong:

In [ ]:
from llama_index.core.llms import ChatMessage
from llama_index.llms.openai import OpenAI

kong_url = "http://127.0.0.1:8000"
kong_route = "gpt-4o"

llm = OpenAI(model=kong_route, api_base=f'{kong_url}/{kong_route}', api_key="department-1-api-key")

messages = [
    ChatMessage(
        role="system", content="You are a mathematician."
    ),
    ChatMessage(role="user", content="What are you?"),
]

response = llm.chat(messages)
print(response)

#### Custom Tool Usage

Kong also supports custom tools, defined via any supported OpenAI-compatible SDK, including LlamaIndex.

With the same `kong.yaml` configuration, you can execute a simple custom tool definition:

In [ ]:
from pydantic import BaseModel
from llama_index.core.tools import FunctionTool

class Answer(BaseModel):
    """A result of database lookup"""
    result: str
    remaining: int

def lookup_stock(product: str) -> Answer:
    """Lookup stock in product database."""
    return Answer(result="IN_STOCK", remaining=4)

tool = FunctionTool.from_defaults(fn=lookup_stock)

from llama_index.core.llms import ChatMessage
from llama_index.llms.openai import OpenAI

kong_url = "http://127.0.0.1:8000"
kong_route = "gpt-4o"

llm = OpenAI(model=kong_route, api_base=f'{kong_url}/{kong_route}', api_key="department-1-api-key", strict=True)

response = llm.predict_and_call(
    [tool],
    "Is the New Phone in stock? How many are left?",
)
print(response)